# 향상된 데이터 분석 에이전트
## 주택 가격 예측 데이터셋 분석

이 노트북은 LangChain의 pandas 데이터프레임 에이전트를 사용하여 주택 가격 데이터를 분석합니다.

In [ ]:
# 필요한 라이브러리 설치
# !pip install langchain langchain-experimental langchain-openai pandas matplotlib seaborn numpy scikit-learn python-dotenv

In [ ]:
# 환경 변수에서 API 키 가져오기
import os
import warnings
from dotenv import load_dotenv

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print(" OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인해주세요.")
else:
    print("OpenAI API 키가 성공적으로 로드되었습니다.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# pandas 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [ ]:
# 데이터 로드
try:
    train = pd.read_csv('./data/house/train.csv')
    test = pd.read_csv('./data/house/test.csv')
    print(f"훈련 데이터 로드 완료: {train.shape}")
    print(f"테스트 데이터 로드 완료: {test.shape}")
except FileNotFoundError:
    print("데이터 파일을 찾을 수 없습니다. './data/' 폴더에 train.csv와 test.csv 파일이 있는지 확인해주세요.")
    # 임시로 샘플 데이터 생성
    train = pd.DataFrame({
        'Id': range(1, 101),
        'LotArea': np.random.randint(5000, 20000, 100),
        'OverallQual': np.random.randint(1, 11, 100),
        'GrLivArea': np.random.randint(800, 3000, 100),
        'SalePrice': np.random.randint(100000, 500000, 100)
    })
    test = train.drop('SalePrice', axis=1).head(50)
    print("샘플 데이터를 생성했습니다.")

In [ ]:
# 기본 데이터 정보 확인
print("=== 훈련 데이터 기본 정보 ===")
print(train.info())
print(f"데이터 크기: {train.shape}")
print(f"결측값 총 개수:\n {train.isnull().sum()}")
print(f"중복된 행: {train.duplicated().sum()}")


In [ ]:
for idx, col_name in enumerate(train.columns) :
    missingValueRate = train[col_name].isna().sum() / len(train) * 100
    if missingValueRate > 0 :
        print(f"{idx} {col_name} null rate: {round(missingValueRate, 2)}%")

In [ ]:

print("\n=== 테스트 데이터 기본 정보 ===")
print(test.info())
print(f"데이터 크기: {test.shape}")
print(f"결측값 총 개수: {test.isnull().sum()}")
print(f"중복된 행: {test.duplicated().sum()}")


In [ ]:
for idx, col_name in enumerate(test.columns) :
    missingValueRate = test[col_name].isna().sum() / len(test) * 100
    if missingValueRate > 0 :
        print(f"{idx} {col_name} null rate: {round(missingValueRate, 2)}%")

In [ ]:

# 공통 컬럼과 차이점 확인
train_cols = set(train.columns)
test_cols = set(test.columns)
common_cols = train_cols.intersection(test_cols)
train_only = train_cols - test_cols
test_only = test_cols - train_cols

print(f"\n=== 컬럼 비교 ===")
print(f"공통 컬럼: {len(common_cols)}개")
print(f"훈련 데이터에만 있는 컬럼: {train_only}")
print(f"테스트 데이터에만 있는 컬럼: {test_only}")

In [ ]:
# 데이터 전처리 함수
def preprocess_data(df, is_train=True):
    """데이터 전처리 함수"""
    df_processed = df.copy()
    
    # 수치형 컬럼의 결측값을 중앙값으로 채우기
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_processed[col].isnull().sum() > 0:
            df_processed[col].fillna(df_processed[col].median(), inplace=True)
    
    # 범주형 컬럼의 결측값을 'Unknown'으로 채우기
    categorical_cols = df_processed.select_dtypes(include=[object]).columns
    for col in categorical_cols:
        if df_processed[col].isnull().sum() > 0:
            df_processed[col].fillna('Unknown', inplace=True)
    
    return df_processed

# 데이터 전처리 적용
train_processed = preprocess_data(train, is_train=True)
test_processed = preprocess_data(test, is_train=False)

print("데이터 전처리가 완료되었습니다.")
print(f"훈련 데이터 결측값: {train_processed.isnull().sum().sum()}")
print(f"테스트 데이터 결측값: {test_processed.isnull().sum().sum()}")

In [ ]:
# LangChain 에이전트 설정
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

# LLM 초기화
llm = ChatOpenAI(
    #model='gpt-3.5-turbo-0125', 
    model='gpt-4o-mini',
    temperature=0,
    api_key=OPENAI_API_KEY
)

# 훈련 데이터용 에이전트 생성
train_agent = create_pandas_dataframe_agent(
    llm,
    train_processed,
    #agent_type="openai-tools",
    agent_type="tool-calling",
    verbose=True,
    return_intermediate_steps=False,
    allow_dangerous_code=True,
    prefix="""당신은 주택 가격 예측 데이터셋을 분석하는 전문가입니다. 
    데이터프레임의 이름은 'df'입니다. 한국어로 답변해주세요.
    시각화를 요청받으면 matplotlib를 사용하여 깔끔하고 이해하기 쉬운 그래프를 만들어주세요."""
)

# 테스트 데이터용 에이전트 생성
test_agent = create_pandas_dataframe_agent(
    llm,
    test_processed,
    agent_type="openai-tools",
    verbose=False,
    return_intermediate_steps=False,
    allow_dangerous_code=True,
    prefix="""당신은 테스트 데이터셋을 분석하는 전문가입니다. 
    데이터프레임의 이름은 'df'입니다. 한국어로 답변해주세요."""
)

print("LangChain 에이전트가 성공적으로 생성되었습니다.")

## 기본 데이터 탐색

In [ ]:
# 기본 통계 정보 확인
response = train_agent.invoke('데이터의 기본 통계 정보를 보여주세요. 특히 SalePrice에 대한 정보를 자세히 설명해주세요.')
print(response['output'])

In [ ]:
# 주택 가격 분포 시각화
response = train_agent.invoke("""
SalePrice의 분포를 다음과 같이 시각화해주세요:
1. 히스토그램 (bins=50)
2. 박스플롯
3. 기본 통계량 출력

그래프는 한국어 제목과 라벨을 사용하고, 서브플롯으로 구성해주세요.
""")
print(response['output'])

In [ ]:
# 결측값 분석
response = train_agent.invoke("""
결측값이 많은 상위 10개 컬럼을 찾아서 막대그래프로 시각화해주세요.
결측값 비율도 함께 계산해서 보여주세요.
""")
print(response['output'])

## 상관관계 분석

In [ ]:
# SalePrice와 상관관계가 높은 변수들 찾기
response = train_agent.invoke("""
SalePrice와 상관관계가 높은 상위 10개 수치형 변수를 찾아주세요.
상관계수를 막대그래프로 시각화하고, 각 변수에 대한 간단한 설명도 포함해주세요.
""")
print(response['output'])

In [ ]:
# 주요 변수들 간의 산점도 매트릭스
response = train_agent.invoke("""
SalePrice와 상관관계가 높은 상위 5개 변수들과 SalePrice를 포함하여
pairplot이나 산점도 매트릭스를 만들어주세요.
seaborn을 사용하면 더 좋습니다.
""")
print(response['output'])

## 범주형 변수 분석

In [ ]:
# 주요 범주형 변수들의 분포
response = train_agent.invoke("""
다음 범주형 변수들의 분포를 시각화해주세요:
1. Neighborhood (동네별 주택 개수)
2. OverallQual (전체 품질별 분포)
3. SaleType (판매 유형별 분포)

각각을 서브플롯으로 만들고, 가독성을 위해 적절히 회전시켜주세요.
""")
print(response['output'])

In [ ]:
# 범주형 변수별 평균 주택 가격
response = train_agent.invoke("""
Neighborhood별 평균 SalePrice를 계산하고 막대그래프로 시각화해주세요.
가격 순으로 정렬하고, 가장 비싸거나 저렴한 동네들을 분석해주세요.
""")
print(response['output'])

## 고급 분석

In [ ]:
# 연도별 주택 가격 트렌드
response = train_agent.invoke("""
YearBuilt(건축연도)와 YrSold(판매연도)에 따른 주택 가격의 트렌드를 분석해주세요.
시계열 그래프로 시각화하고, 주요 인사이트를 설명해주세요.
""")
print(response['output'])

In [ ]:
# 이상치 탐지
response = train_agent.invoke("""
SalePrice에서 이상치(outliers)를 탐지하고 분석해주세요.
IQR 방법을 사용하여 이상치를 식별하고, 이상치들의 특성을 분석해주세요.
박스플롯도 함께 그려주세요.
""")
print(response['output'])

## 테스트 데이터 분석

In [ ]:
# 테스트 데이터 기본 정보
response = test_agent.invoke("""
테스트 데이터의 기본 정보를 분석해주세요:
1. 데이터 형태와 크기
2. 결측값 현황
3. 주요 수치형 변수들의 기본 통계량
""")
print(response['output'])

In [ ]:
# 훈련 데이터와 테스트 데이터 비교
print("=== 훈련 데이터와 테스트 데이터 분포 비교 ===")

# 공통 수치형 변수들 선택
common_numeric_cols = ['LotArea', 'OverallQual', 'YearBuilt', 'GrLivArea', 'GarageCars']
available_cols = [col for col in common_numeric_cols if col in train_processed.columns and col in test_processed.columns]

if available_cols:
    fig, axes = plt.subplots(len(available_cols), 1, figsize=(12, 4*len(available_cols)))
    if len(available_cols) == 1:
        axes = [axes]
    
    for i, col in enumerate(available_cols):
        axes[i].hist(train_processed[col], alpha=0.5, label='Train', bins=30, density=True)
        axes[i].hist(test_processed[col], alpha=0.5, label='Test', bins=30, density=True)
        axes[i].set_title(f'{col} Distribution Comparison')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Density')
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("비교할 수 있는 공통 수치형 변수가 없습니다.")

## 간단한 예측 모델

In [ ]:
# 간단한 머신러닝 모델 구축
def build_simple_model():
    """간단한 랜덤포레스트 모델 구축"""
    
    # 수치형 변수만 선택 (SalePrice 제외)
    if 'SalePrice' in train_processed.columns:
        numeric_features = train_processed.select_dtypes(include=[np.number]).columns.drop(['Id', 'SalePrice'])
        
        X = train_processed[numeric_features]
        y = train_processed['SalePrice']
        
        # 훈련/검증 분할
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # 모델 훈련
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)
        
        # 예측 및 평가
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        
        print(f"모델 성능:")
        print(f"RMSE: {np.sqrt(mse):,.2f}")
        print(f"R² Score: {r2:.4f}")
        
        # 특성 중요도
        feature_importance = pd.DataFrame({
            'feature': numeric_features,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        # 특성 중요도 시각화
        plt.figure(figsize=(10, 8))
        plt.barh(feature_importance.head(15)['feature'], feature_importance.head(15)['importance'])
        plt.title('Top 15 Feature Importance')
        plt.xlabel('Importance')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        
        return model, feature_importance
    else:
        print("SalePrice 컬럼이 없어서 모델을 구축할 수 없습니다.")
        return None, None

model, feature_importance = build_simple_model()

## 종합 인사이트

In [ ]:
# 최종 인사이트 요청
response = train_agent.invoke("""
지금까지의 분석을 바탕으로 주택 가격에 영향을 미치는 주요 요인들을 요약해주세요.
다음 내용을 포함해주세요:
1. 가격에 가장 큰 영향을 미치는 상위 5개 요인
2. 주목할 만한 패턴이나 트렌드
3. 데이터의 특이사항이나 주의점
4. 추가 분석이 필요한 영역
""")
print(response['output'])

## 추가 분석 함수들

In [ ]:
# 사용자 정의 분석 함수
def custom_analysis(question):
    """사용자 정의 질문으로 분석 수행"""
    try:
        response = train_agent.invoke(question)
        return response['output']
    except Exception as e:
        return f"분석 중 오류가 발생했습니다: {str(e)}"

def compare_datasets(question):
    """훈련 데이터와 테스트 데이터 비교 분석"""
    print("=== 훈련 데이터 분석 ===")
    train_response = train_agent.invoke(question)
    print(train_response['output'])
    
    print("\n=== 테스트 데이터 분석 ===")
    test_response = test_agent.invoke(question)
    print(test_response['output'])

# 예시 사용법
print("✅ 추가 분석 함수들이 준비되었습니다.")
print("\n사용 예시:")
print("custom_analysis('특정 동네의 주택 가격 분포를 분석해주세요')")
print("compare_datasets('OverallQual의 분포를 분석해주세요')")

In [ ]:
# 대화형 분석 예시
print("=== 대화형 분석 예시 ===")
print("\n원하는 분석을 자유롭게 요청해보세요!")
print("예: '방 개수와 주택 가격의 관계를 분석해주세요'")
print("예: '가장 비싼 주택들의 특징을 찾아주세요'")
print("예: '건축 연도별 주택 가격 변화를 시각화해주세요'")

# 여기서부터는 사용자가 직접 질문을 입력할 수 있습니다.
# 예시:
# result = custom_analysis('방 개수와 주택 가격의 관계를 분석해주세요')
# print(result)